# Notebook per regressione lineare

Di seguito:
- verrà implementato l'algoritmo di regressione logistica

## Scaricamento dei dati

In [ ]:
# File per scaricare i dati per fare analisi di machine learning

import requests
import zipfile
import io
import os

# List of URLs to your zipped files on AWS
urls = [
    "https://phm-datasets.s3.amazonaws.com/Data_Challenge_PHM2024_training_data.zip"]

# Directory to save the extracted files
output_dir = "dataset"
os.makedirs(output_dir, exist_ok=True)

for url in urls:
    try:
        print(f"Downloading {url}...")
        response = requests.get(url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes

        # Read the zip file from the response content
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            # Extract all contents to the specified output directory
            zip_ref.extractall(output_dir)
            print(f"Extracted files from {url} to {output_dir}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
    except zipfile.BadZipFile:
        print(f"Error: The downloaded file from {url} is not a valid zip file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

print("Download and extraction complete.")

# Now you can access your CSV files in the 'downloaded_data' directory
# For example, to list the files in the directory:
import glob
csv_files = glob.glob(os.path.join(output_dir, "*.csv"))
print("CSV files found:", csv_files)

## Estrazione dei dati

In [ ]:
import pandas as pd

def load_data(x_path, y_path):
  """
  Loads X.csv and the second column of y.csv into a single pandas DataFrame.

  Args:
    x_path (str): The path to the X.csv file.
    y_path (str): The path to the y.csv file.

  Returns:
    pandas.DataFrame: A DataFrame containing the data from X.csv
                      and the second column of y.csv.
  """
  x = pd.read_csv(x_path)
  y = pd.read_csv(y_path)

  # Assuming y has at least 2 columns and the second column is at index 1
  if y.shape[1] > 1:
    combined_data = x.copy()
    combined_data['y_target'] = y.iloc[:, 1]
    return combined_data
  else:
    print("Error: y.csv does not have a second column.")
    return x

# Example usage:
# Assuming your files are in the 'dataset' directory as per the preceding code
x_path = 'dataset/X_train.csv'
y_path = 'dataset/y_train.csv'

data = load_data(x_path, y_path)

# You can now work with the 'data' DataFrame
print(data.head())


## Creazione training-set testing-set

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

print("Dimensione data_train:", data_train.shape)
print("Dimensione data_test:", data_test.shape)

In [ ]:
data_test

# PIPELINE

## Cambio nome delle feature

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        if not isinstance(column_names, list):
            raise TypeError("column_names must be a list.")
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        is_tuple = isinstance(X, tuple)
        if is_tuple:
            if not X:
                raise ValueError("Input tuple is empty.")
            df = X[0]
            rest = X[1:]
        else:
            df = X
            rest = ()

        if not isinstance(df, pd.DataFrame):
            raise TypeError("Input (or first element if tuple) must be a DataFrame.")

        # Drop 'id' column if present
        if 'id' in df.columns:
            df = df.drop(columns='id')

        # Check column count matches
        if len(self.column_names) != df.shape[1]:
            if len(self.column_names) + 1 == df.shape[1] and 'faulty' in df.columns:
                print("Detected 'faulty' column, adjusting rename.")
                new_cols = self.column_names + ['faulty']
                if len(new_cols) != df.shape[1]:
                    raise ValueError("Mismatch in column count with 'faulty' included.")
                df_renamed = df.copy()
                df_renamed.columns = new_cols
            else:
                raise ValueError("Column count mismatch after dropping 'id'.")
        else:
            df_renamed = df.copy()
            df_renamed.columns = self.column_names

        return (df_renamed,) + rest if rest else df_renamed

# ---------------------- Setup ----------------------
new_names_for_train_data = [
    'torque_meas', 'outside_air_temp', 'mean_gas_temp',
    'power_avail', 'indicated_air_speed', 'net_power',
    'compressor_speed', 'health_state'
]

# ---------------------- Pipeline ----------------------
pipeline_with_renaming = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data))
])

# Apply pipeline
data_train_renamed = pipeline_with_renaming.fit_transform(data_train)

# ---------------------- Output ----------------------
print("Renamed DataFrame (without 'id'):")
print(data_train_renamed.head())

## Standardizzazione

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def standardize_columns(df, columns_to_standardize):
  """
  Standardizes specified columns of a pandas DataFrame to have values between 0 and 1
  using MinMaxScaler.

  Args:
    df: The pandas DataFrame to standardize.
    columns_to_standardize: A list of column names to standardize.

  Returns:
    The DataFrame with the specified columns standardized.
  """
  scaler = MinMaxScaler()
  df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])
  return df

# Example usage:
# Assuming you want to standardize all numerical columns except the index and the target variable
# Identify numerical columns (excluding 'idx' and 'health_state' in this case)
numerical_cols = data_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
columns_to_standardize = [col for col in numerical_cols if col not in ['idx', 'health_state']]

print(f"\nColumns to standardize: {columns_to_standardize}")

data_train_standardized = standardize_columns(data_train.copy(), columns_to_standardize)

print("\nDataFrame after standardization:")
print(data_train_standardized.head())
print("\nDescriptive statistics after standardization:")
print(data_train_standardized.describe())

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che contenga la funzione transform_with_custom_root standardize_columns

class ColumnStandardizer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_standardize=None):
        self.columns_to_standardize = columns_to_standardize
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a DataFrame.")

        if self.columns_to_standardize is None:
            # Identify numerical columns excluding the target if present
            numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
            # Assuming 'y_target' or similar is the target and should not be scaled
            # This part might need adjustment based on how your pipeline handles the target column
            self._cols_to_fit = [col for col in numerical_cols if col not in ['y_target', 'health_state']] # Exclude common target/index names
        else:
             if not isinstance(self.columns_to_standardize, (list, tuple)):
                 raise TypeError("columns_to_standardize must be a list or tuple of column names.")
             # Ensure all specified columns exist in the input DataFrame
             missing_cols = [col for col in self.columns_to_standardize if col not in X.columns]
             if missing_cols:
                 raise ValueError(f"Columns not found in input DataFrame: {missing_cols}")
             self._cols_to_fit = self.columns_to_standardize

        if not self._cols_to_fit:
            print("Warning: No columns selected for standardization.")
            return self

        # Fit the scaler only on the selected numerical columns
        self.scaler.fit(X[self._cols_to_fit])
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a DataFrame.")
        if not hasattr(self, '_cols_to_fit'):
             raise RuntimeError("The transformer has not been fitted yet.")

        df_standardized = X.copy()

        if not self._cols_to_fit:
             return df_standardized # Return original if no columns to standardize

        # Ensure that the columns to transform are actually present in the input DataFrame
        present_cols_to_transform = [col for col in self._cols_to_fit if col in df_standardized.columns]

        if not present_cols_to_transform:
             print("Warning: None of the specified columns for standardization are present in the input DataFrame.")
             return df_standardized

        # Transform only the present columns
        df_standardized[present_cols_to_transform] = self.scaler.transform(df_standardized[present_cols_to_transform])

        return df_standardized


## Rimozione colonne

In [ ]:
# prompt: crea una trasformazione che droppi certe colonne di un dataset

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        if not isinstance(columns_to_drop, (list, tuple)):
            raise TypeError("columns_to_drop must be a list or tuple of column names.")
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        # Nothing to fit, but check if columns exist in the input
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a DataFrame.")
        missing_cols = [col for col in self.columns_to_drop if col not in X.columns]
        if missing_cols:
            print(f"Warning: Columns to drop not found in input DataFrame: {missing_cols}. These will be ignored during transform.")
            # Store the columns that are actually present to avoid errors during transform
            self._cols_to_drop_present = [col for col in self.columns_to_drop if col in X.columns]
        else:
            self._cols_to_drop_present = self.columns_to_drop
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a DataFrame.")
        if not hasattr(self, '_cols_to_drop_present'):
             raise RuntimeError("The transformer has not been fitted yet.")

        df_dropped = X.copy()
        # Drop only the columns that were found during fit
        df_dropped = df_dropped.drop(columns=self._cols_to_drop_present, errors='ignore')
        return df_dropped

## Radice custom

### Trasformazione in work_with_data

In [ ]:
def transform_with_custom_root(df, column_name, root_degree):
  """
  Applies a custom root transformation (1/root_degree power) to a column.
  Handles positive, negative, and zero values appropriately based on the root degree.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to transform.
    root_degree (float): The degree of the root (e.g., 2 for square root, 3 for cube root).

  Returns:
    pd.DataFrame: The DataFrame with the transformed column.
  """
  new_column_name = f'{column_name}_root_{root_degree:.2f}_transformed'

  if root_degree == 0:
      raise ValueError("Root degree cannot be zero.")
  elif root_degree % 2 == 0:  # Even root
      # For even roots, we can only take the root of non-negative numbers
      if (df[column_name] < 0).any():
          print(f"Warning: Column '{column_name}' contains negative values. Cannot apply even root directly.")
          # You might choose to handle this by taking the root of the absolute value,
          # or setting negative values to NaN, depending on your data context.
          # Here, we'll take the root of the absolute value for demonstration.
          df[new_column_name] = np.power(np.abs(df[column_name]), 1/root_degree)
      else:
          df[new_column_name] = np.power(df[column_name], 1/root_degree)
  else:  # Odd root
      # Odd roots can handle positive, negative, and zero values
      df[new_column_name] = np.sign(df[column_name]) * np.power(np.abs(df[column_name]), 1/root_degree)

  return df

### Trasformazione pipeline

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che contenga la funzione transform_with_custom_root

class CustomRootTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, root_transformations=None):
        self.root_transformations = root_transformations

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        df_transformed = X.copy()

        if self.root_transformations:
            for col, deg in self.root_transformations.items():
                if col not in df_transformed.columns:
                    # print(f"Warning: Column '{col}' not found in DataFrame. Skipping root transformation.") # Silence this
                    continue

                # print(f"Applying custom root {deg} transformation to column '{col}'...") # Silence this
                # Use the existing transform_with_custom_root function but handle its potential print
                # A more robust way would be to modify transform_with_custom_root itself
                # For this case, we assume transform_with_custom_root doesn't print excessively
                # or we accept its prints for now. To strictly silence, you'd need to redirect stdout.
                df_transformed = transform_with_custom_root(df_transformed, col, deg)

        return df_transformed



## Quantile transformer

### Trasformazione in work_with_data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer

def transform_with_quantile_transformer(df, column_name, output_distribution='normal'):
  """
  Applies a quantile transformation to a specified column to make its distribution
  more Gaussian-like (normal) or uniform.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to transform.
    output_distribution (str): The desired output distribution. 'normal'
                               for Gaussian-like, 'uniform' for uniform.
                               Default is 'normal'.

  Returns:
    pd.DataFrame: The DataFrame with the transformed column.
  """
  if column_name not in df.columns:
    raise ValueError(f"La colonna '{column_name}' non è presente nel DataFrame.")

  # Reshape the data as the transformer expects a 2D array
  data_to_transform = df[column_name].values.reshape(-1, 1)

  # Initialize and fit the QuantileTransformer
  quantile_transformer = QuantileTransformer(output_distribution=output_distribution)
  transformed_data = quantile_transformer.fit_transform(data_to_transform)

  # Add the transformed data back to the DataFrame with a new column name
  df[f'{column_name}_quantile_transformed'] = transformed_data

  return df

# Apply the quantile transformation to the 'power_avail' column
#data_train = transform_with_quantile_transformer(data_train.copy(), 'power_avail', output_distribution='normal')



### Trasformazione pipeline

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che contenga la funzione transform_with_quantile_transformer

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import QuantileTransformer

class QuantileTransformerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_transform, output_distribution='normal'):
        """
        Initializes the QuantileTransformerWrapper.

        Args:
          columns_to_transform (list): A list of column names to transform.
          output_distribution (str): The desired output distribution. 'normal'
                                     for Gaussian-like, 'uniform' for uniform.
                                     Default is 'normal'.
        """
        if not isinstance(columns_to_transform, (list, tuple)):
            raise TypeError("columns_to_transform must be a list or tuple of column names.")
        self.columns_to_transform = columns_to_transform
        self.output_distribution = output_distribution
        self.transformers = {} # Dictionary to store transformers for each column

    def fit(self, X, y=None):
        """
        Fits a QuantileTransformer for each specified column.

        Args:
          X (pd.DataFrame): The input DataFrame.
          y: Ignored.

        Returns:
          self: The fitted transformer.
        """
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        for col in self.columns_to_transform:
            if col not in X.columns:
                print(f"Warning: Column '{col}' not found in DataFrame during fit. Skipping.")
                continue

            # Initialize and fit the QuantileTransformer for the current column
            transformer = QuantileTransformer(output_distribution=self.output_distribution)
            transformer.fit(X[col].values.reshape(-1, 1))
            self.transformers[col] = transformer

        return self

    def transform(self, X):
        """
        Applies the fitted QuantileTransformer to the specified columns.

        Args:
          X (pd.DataFrame): The input DataFrame.

        Returns:
          pd.DataFrame: The DataFrame with the transformed columns.
        """
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
        if not hasattr(self, 'transformers'):
            raise RuntimeError("The transformer has not been fitted yet.")

        df_transformed = X.copy()

        for col, transformer in self.transformers.items():
            if col in df_transformed.columns:
                # Apply the transformation
                transformed_data = transformer.transform(df_transformed[col].values.reshape(-1, 1))
                # Add the transformed data back to the DataFrame with a new column name
                df_transformed[f'{col}_quantile_transformed'] = transformed_data
            else:
                 print(f"Warning: Column '{col}' not found in DataFrame during transform. Skipping.")


        return df_transformed


## Binary transformation

### Trasformazione work_with_data

In [ ]:
def create_binned_qualitative_variable(df, column_name, num_bins, strategy='quantile'):
  """
  Creates a qualitative (categorical) variable by binning a numerical column.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the numerical column to bin.
    num_bins (int): The desired number of bins.
    strategy (str): The strategy to use for binning. 'quantile' uses quantiles
                    to ensure bins have approximately equal numbers of observations.
                    'uniform' creates bins with equal widths. Default is 'quantile'.

  Returns:
    pd.DataFrame: The DataFrame with a new qualitative column.
                  The new column name will be f'{column_name}_binned_{num_bins}_{strategy}'.
  """
  if column_name not in df.columns:
    raise ValueError(f"La colonna '{column_name}' non è presente nel DataFrame.")
  if num_bins <= 1:
      raise ValueError("Il numero di bins deve essere maggiore di 1.")

  new_column_name = f'{column_name}_binned_{num_bins}_{strategy}'

  if strategy == 'quantile':
    # Use qcut to create bins based on quantiles (approximately equal number of observations)
    # `duplicates='drop'` handles cases where quantile boundaries are not unique,
    # which can happen with skewed or discrete data.
    df[new_column_name] = pd.qcut(df[column_name], q=num_bins, labels=False, duplicates='drop')
  elif strategy == 'uniform':
    # Use cut to create bins of equal width
    df[new_column_name] = pd.cut(df[column_name], bins=num_bins, labels=False, include_lowest=True)
  else:
    raise ValueError(f"Strategia di binning non valida: '{strategy}'. Scegliere tra 'quantile' o 'uniform'.")

  # Convert the binned column to object/category type if needed, or keep as int for simplicity
  # Here we keep it as int representing the bin number

  return df

### Trasformazione pipeline

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che contenga la funzione  create_binned_qualitative_variable

class BinnedQualitativeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, binning_config=None):
        self.binning_config = binning_config

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        df_transformed = X.copy()

        if self.binning_config:
            for col, (num_bins, strategy) in self.binning_config.items():
                if col not in df_transformed.columns:
                    # print(f"Warning: Column '{col}' not found in DataFrame. Skipping binning transformation.") # Silence this
                    continue

                # print(f"Applying binning transformation to column '{col}' with {num_bins} bins and strategy '{strategy}'...") # Silence this
                df_transformed = create_binned_qualitative_variable(df_transformed, col, num_bins, strategy)

        return df_transformed


## PCA Compressor speed net power

### Trasformazione con work_with_data

In [ ]:

#features_for_pca = data_train[['compressor_speed', 'net_power']]


#pca = PCA(n_components=1)

#data_train['compressor_speed_net_power_pca'] = pca.fit_transform(features_for_pca)

### Trasformazione pipeline

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che  come nella cella sopra a partire dalle feature 'compressor_speed' e net_power' tramite il metodo PCA crei una nuova variabile 'compressor_speed_net_power_pca', droppa poi le feature 'net_power' e 'compressor_speed'

class PCATransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_pca, n_components=1, new_column_prefix="pca"):
        if not isinstance(columns_to_pca, list) or len(columns_to_pca) < 1:
            raise ValueError("columns_to_pca must be a list of at least one column name.")
        if n_components < 1 or n_components > len(columns_to_pca):
            raise ValueError("n_components must be between 1 and the number of columns to PCA.")

        self.columns_to_pca = columns_to_pca
        self.n_components = n_components
        self.new_column_prefix = new_column_prefix
        self.pca_model = None
        self.new_column_names = []


    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        missing_cols = [col for col in self.columns_to_pca if col not in X.columns]
        if missing_cols:
            raise ValueError(f"The following columns for PCA were not found in the DataFrame: {missing_cols}")

        # print(f"Fitting PCA on columns: {self.columns_to_pca}") # Silence this
        self.pca_model = PCA(n_components=self.n_components)
        self.pca_model.fit(X[self.columns_to_pca])

        if self.n_components == 1:
            self.new_column_names = [f'{self.new_column_prefix}_{"_".join(self.columns_to_pca).replace(".", "_")}']
        else:
             self.new_column_names = [f'{self.new_column_prefix}_{i+1}' for i in range(self.n_components)]


        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        if self.pca_model is None:
            raise RuntimeError("PCA model has not been fitted yet. Call fit() first.")

        df_transformed = X.copy()

        cols_to_drop_exist = [col for col in self.columns_to_pca if col in df_transformed.columns]

        # print(f"Transforming columns {self.columns_to_pca} using PCA...") # Silence this
        pca_result = self.pca_model.transform(df_transformed[self.columns_to_pca])

        if self.n_components == 1:
             df_transformed[self.new_column_names[0]] = pca_result[:, 0]
        else:
            for i, col_name in enumerate(self.new_column_names):
                df_transformed[col_name] = pca_result[:, i]

        # print(f"Dropping original PCA columns: {cols_to_drop_exist}") # Silence this
        df_transformed = df_transformed.drop(columns=cols_to_drop_exist)

        return df_transformed

## Feature engineering

### Trasformazione work_with_data

In [ ]:
#data_train['torque_times_temp'] = data_train['torque_meas'] * data_train['outside_air_temp']#

### Trasformazione pipeline

## torque_meas e outside_air_speed

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che,  come nella cella sopra ,faccia il prodotto dalle feature 'torque_meas' e outside_air_temp' , e crei  una nuova feature 'torque_times_temp', droppa poi le feature ''torque_meas' e 'outside_air_temp


class TorqueTempFeature(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        required_cols = ['torque_meas', 'outside_air_temp']
        missing_cols = [col for col in required_cols if col not in X.columns]
        if missing_cols:
            raise ValueError(f"The following required columns for TorqueTempFeature were not found in the DataFrame: {missing_cols}")
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        df_transformed = X.copy()

        required_cols = ['torque_meas', 'outside_air_temp']
        missing_cols = [col for col in required_cols if col not in df_transformed.columns]

        if missing_cols:
             # print(f"Warning: Skipping 'torque_times_temp' creation as columns are missing: {missing_cols}") # Silence this
             return df_transformed
        else:
            # print("Creating 'torque_times_temp' feature...") # Silence this
            df_transformed['torque_times_temp'] = df_transformed['torque_meas'] * df_transformed['outside_air_temp']
            # print("Dropping 'torque_meas' and 'outside_air_temp' columns...") # Silence this
            df_transformed = df_transformed.drop(columns=['torque_meas', 'outside_air_temp'])

        return df_transformed

## torque_meas e mean_gas_temp

In [ ]:
# prompt: crea una classe che sia compatibile con l'oggetto pipeline di scikit che,  come nella cella sopra ,faccia il prodotto dalle feature 'torque_meas' e mean_gas_temp' , e crei  una nuova feature 'torque_times_temp', droppa poi le feature ''torque_meas' e 'mean_gas_temp


class TorqueTempFeature2(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        required_cols = ['torque_meas', 'mean_gas_temp']
        missing_cols = [col for col in required_cols if col not in X.columns]
        if missing_cols:
            raise ValueError(f"The following required columns for TorqueTempFeature were not found in the DataFrame: {missing_cols}")
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        df_transformed = X.copy()

        required_cols = ['torque_meas', 'mean_gas_temp']
        missing_cols = [col for col in required_cols if col not in df_transformed.columns]

        if missing_cols:
             # print(f"Warning: Skipping 'torque_times_temp' creation as columns are missing: {missing_cols}") # Silence this
             return df_transformed
        else:
            # print("Creating 'torque_times_temp' feature...") # Silence this
            df_transformed['torque_times_temp'] = df_transformed['torque_meas'] * df_transformed['mean_gas_temp']
            # print("Dropping 'torque_meas' and 'mean_gas_temp' columns...") # Silence this
            df_transformed = df_transformed.drop(columns=['torque_meas', 'mean_gas_temp'])

        return df_transformed

## Separo y_target da test set e train set

In [ ]:
# prompt: estrai dal dataframe 'data_train' 'helath state' e salvalo in data_train_y e droppalo in data_train

data_train_y = data_train['y_target']
data_train_x = data_train.drop('y_target', axis=1)

data_test_y = data_test['y_target']
data_test_x = data_test.drop('y_target', axis=1)

print("data_train after dropping 'y_target':")
print(data_train.head())
print("\ndata_train_y (extracted 'y_target'):")
print(data_train_y.head())


# Creazione Pipeline

## Prima Pipeline

In [ ]:
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

# Update the pipeline to include the new transformations
preprocessing = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    ('custom_root_power_avail', CustomRootTransformer(root_transformations={'power_avail': 2.35})),
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    ('torque_temp_feature', TorqueTempFeature()),
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail'])), # Add ColumnDropper here
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'mean_gas_temp',
        'pca_net_power_compressor_speed',
        'torque_times_temp',
        'power_avail_root_2.35_transformed'])),
  # Assuming you still want scaling at the end
])

# Apply the updated pipeline to data_train_x
data_train_processed_x = preprocessing.fit_transform(data_train_x)

data_train_processed_x

## Applicando anche il Onehotencoding

In [ ]:
# prompt: prendi esempio dalla cella sopra m, ma esegui anche one_hot_encoding su indicated_air_speed_binned_5_quantile

import pandas as pd
# Update the pipeline to include the new transformations and OneHotEncoder
preprocessing_OH = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    ('custom_root_power_avail', CustomRootTransformer(root_transformations={'power_avail': 2.35})),
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    ('torque_temp_feature', TorqueTempFeature()),
    # The ColumnDropper should now drop the original 'indicated_air_speed' and 'power_avail'
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail'])),
    # Standardize numerical columns (including the new ones)
    # Note: 'indicated_air_speed_binned_5_quantile' is categorical and will be handled by OneHotEncoder
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'mean_gas_temp',
        'pca_net_power_compressor_speed',
        'torque_times_temp',
        'power_avail_root_2.35_transformed'])),
    # Apply OneHotEncoder to the binned categorical column
    ('one_hot_encode_binned_air_speed', ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['indicated_air_speed_binned_5_quantile']) # sparse_output=False to get a dense array
        ],
        remainder='passthrough' # Keep other columns (the scaled numerical ones)
    ))
])

# Apply the updated pipeline to data_train_x
# The output of ColumnTransformer is a numpy array when remainder='passthrough'
data_train_processed_x_array_OH = preprocessing_OH.fit_transform(data_train_x)






In [ ]:
# prompt: trasforma i dati di data_train_processed_x_array_OH in un  data_frame con i nomi delle feature corrispondenti non printare

import pandas as pd
def to_dataframe(preprocessing, data_train_x):


  data_train_processed_x_array = preprocessing.fit_transform(data_train_x)


  column_transformer_step = preprocessing.named_steps['one_hot_encode_binned_air_speed']


  pipeline_before_onehot = Pipeline(preprocessing.steps[:-1])


  df_before_onehot = pipeline_before_onehot.fit_transform(data_train_x)


  onehot_cols_to_encode = column_transformer_step.transformers[0][2]


  remainder_cols = [col for col in df_before_onehot.columns if col not in onehot_cols_to_encode]


  fitted_onehot_transformer = column_transformer_step.named_transformers_['onehot']
  onehot_feature_names = fitted_onehot_transformer.get_feature_names_out(onehot_cols_to_encode)


  all_feature_names = list(onehot_feature_names) + remainder_cols

  data_train_processed = pd.DataFrame(data_train_processed_x_array, columns=all_feature_names)

  return data_train_processed


data_train_processed_OH = to_dataframe(preprocessing_OH, data_train_x)

data_train_processed_OH.index = data_train_x.index

data_train_processed_OH

## Tenendo outside_air_temp e torque_meas

In [ ]:
# prompt: prendi esempio dalla cella sopra m, ma esegui anche one_hot_encoding su indicated_air_speed_binned_5_quantile

import pandas as pd
# Update the pipeline to include the new transformations and OneHotEncoder
preprocessing_OH_OT = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    ('custom_root_power_avail', CustomRootTransformer(root_transformations={'power_avail': 2.35})),
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    #('torque_temp_feature', TorqueTempFeature()),
    # The ColumnDropper should now drop the original 'indicated_air_speed' and 'power_avail'
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail'])),
    # Standardize numerical columns (including the new ones)
    # Note: 'indicated_air_speed_binned_5_quantile' is categorical and will be handled by OneHotEncoder
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'mean_gas_temp',
        'pca_net_power_compressor_speed',
        'outside_air_temp',
        'torque_meas',
        'power_avail_root_2.35_transformed',
      ])),
    # Apply OneHotEncoder to the binned categorical column
    ('one_hot_encode_binned_air_speed', ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['indicated_air_speed_binned_5_quantile']) # sparse_output=False to get a dense array
        ],
        remainder='passthrough' # Keep other columns (the scaled numerical ones)
    ))
])

# Apply the updated pipeline to data_train_x
# The output of ColumnTransformer is a numpy array when remainder='passthrough'
data_train_processed_x_array_OH_OT = preprocessing_OH_OT.fit_transform(data_train_x)



In [ ]:
# prompt: trasforma i dati di data_train_processed_x_array_OH_OT in un  data_frame con i nomi delle feature corrispondenti non printare

data_train_processed_OH_OT = to_dataframe(preprocessing_OH_OT, data_train_x)

data_train_processed_OH_OT.index = data_train_x.index

data_train_processed_OH_OT

## Quantile transformer su power available

In [ ]:
# prompt: prendi esempio dalla cella sotto ma invece di  CustomRootTransformer usa Quantiletransformer

# Update the pipeline to include the new transformations and OneHotEncoder
preprocessing_OH_QT = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    #('custom_root_power_avail', CustomRootTransformer(root_transformations={'power_avail': 2.35})), # Remove custom root
    ('quantile_power_avail', QuantileTransformerWrapper(columns_to_transform=['power_avail'], output_distribution='normal')), # Add QuantileTransformer
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    # The ColumnDropper should now drop the original 'indicated_air_speed', 'power_avail', 'torque_meas', and 'mean_gas_temp'
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail'])),
    # Standardize numerical columns (including the new ones)
    # Note: 'indicated_air_speed_binned_5_quantile' is categorical and will be handled by OneHotEncoder
    # Note: 'power_avail_quantile_transformed' is now the transformed column
    # Note: 'torque_times_temp' is the new combined feature
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'mean_gas_temp', # Dropped
        'pca_net_power_compressor_speed',
        'outside_air_temp',
        'torque_meas', # Dropped
        #'power_avail_root_2.35_transformed', # Removed
        'power_avail_quantile_transformed', # New transformed column
      ])),
    # Apply OneHotEncoder to the binned categorical column
    ('one_hot_encode_binned_air_speed', ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['indicated_air_speed_binned_5_quantile']) # sparse_output=False to get a dense array
        ],
        remainder='passthrough' # Keep other columns (the scaled numerical ones)
    ))
])

# Apply the updated pipeline to data_train_x
# The output of ColumnTransformer is a numpy array when remainder='passthrough'
data_train_processed_x_array_OH_QT = preprocessing_OH_QT.fit_transform(data_train_x)


In [ ]:
# prompt: trasforma i dati di data_train_processed_x_array_OH_OT in un  data_frame con i nomi delle feature corrispondenti non printare

data_train_processed_OH_QT = to_dataframe(preprocessing_OH_QT, data_train_x)


data_train_processed_OH_QT.index = data_train_x.index


data_train_processed_OH_QT

## Feature engineering of mean_temp e torq

In [ ]:
# prompt: prendi esempio dalla cella sopra m, ma esegui anche one_hot_encoding su indicated_air_speed_binned_5_quantile

import pandas as pd
# Update the pipeline to include the new transformations and OneHotEncoder
preprocessing_OH_CH = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    ('quantile_power_avail', QuantileTransformerWrapper(columns_to_transform=['power_avail'], output_distribution='normal')),
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    ('torque_temp_feature', TorqueTempFeature2()),
    # The ColumnDropper should now drop the original 'indicated_air_speed' and 'power_avail'
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail'])),
    # Standardize numerical columns (including the new ones)
    # Note: 'indicated_air_speed_binned_5_quantile' is categorical and will be handled by OneHotEncoder
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'outside_air_temp',
        'pca_net_power_compressor_speed',
        'torque_times_temp',
        'power_avail_quantile_transformed'])),
    # Apply OneHotEncoder to the binned categorical column
    ('one_hot_encode_binned_air_speed', ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['indicated_air_speed_binned_5_quantile']) # sparse_output=False to get a dense array
        ],
        remainder='passthrough' # Keep other columns (the scaled numerical ones)
    ))
])

# Apply the updated pipeline to data_train_x
# The output of ColumnTransformer is a numpy array when remainder='passthrough'
data_train_processed_x_array_OH_CH = preprocessing_OH_CH.fit_transform(data_train_x)




In [ ]:
# prompt: trasforma i dati di data_train_processed_x_array_OH_OT in un  data_frame con i nomi delle feature corrispondenti non printare

data_train_processed_OH_CH = to_dataframe(preprocessing_OH_CH, data_train_x)

data_train_processed_OH_CH.index = data_train_x.index

data_train_processed_OH_CH

# Due quantile

In [ ]:
# prompt: prendi esempio dalla cella sotto ma invece di  CustomRootTransformer usa Quantiletransformer

# Update the pipeline to include the new transformations and OneHotEncoder
preprocessing_OH_AQT = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data[:-1])), # Exclude 'health_state' if not in data_train_x
    #('custom_root_power_avail', CustomRootTransformer(root_transformations={'power_avail': 2.35})), # Remove custom root
    ('quantile_power_avail', QuantileTransformerWrapper(columns_to_transform=['power_avail','torque_meas'], output_distribution='normal')), # Add QuantileTransformer
    ('binned_air_speed', BinnedQualitativeTransformer(binning_config={'indicated_air_speed': (5, 'quantile')})),
    ('pca_speed_power', PCATransformer(columns_to_pca=['net_power', 'compressor_speed'], n_components=1)),
    # The ColumnDropper should now drop the original 'indicated_air_speed', 'power_avail', 'torque_meas', and 'mean_gas_temp'
    ('drop_columns', ColumnDropper(columns_to_drop=['indicated_air_speed', 'power_avail','torque_meas'])),
    # Standardize numerical columns (including the new ones)
    # Note: 'indicated_air_speed_binned_5_quantile' is categorical and will be handled by OneHotEncoder
    # Note: 'power_avail_quantile_transformed' is now the transformed column
    # Note: 'torque_times_temp' is the new combined feature
    ('scale_data', ColumnStandardizer(columns_to_standardize=[
        'mean_gas_temp',
        'pca_net_power_compressor_speed',
        'outside_air_temp',
        'torque_meas_quantile_transformed', # Dropped
        #'power_avail_root_2.35_transformed', # Removed
        'power_avail_quantile_transformed', # New transformed column
      ])),
    # Apply OneHotEncoder to the binned categorical column
    ('one_hot_encode_binned_air_speed', ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['indicated_air_speed_binned_5_quantile']) # sparse_output=False to get a dense array
        ],
        remainder='passthrough' # Keep other columns (the scaled numerical ones)
    ))
])

# Apply the updated pipeline to data_train_x
# The output of ColumnTransformer is a numpy array when remainder='passthrough'
data_train_processed_x_array_OH_AQT = preprocessing_OH_QT.fit_transform(data_train_x)


In [ ]:
# prompt: trasforma i dati di data_train_processed_x_array_OH_OT in un  data_frame con i nomi delle feature corrispondenti non printare

data_train_processed_OH_AQT = to_dataframe(preprocessing_OH_AQT, data_train_x)


data_train_processed_OH_AQT.index = data_train_x.index


data_train_processed_OH_AQT

# Visualizzazione delle pipeline

## Training and Evaluating on the Training Set

## Senza onehotencoding

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = make_pipeline(preprocessing, LogisticRegression())
log_reg.fit(data_train_x, data_train_y)

## One_hot_encoding

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_OH = make_pipeline(preprocessing_OH, LogisticRegression())
log_reg_OH.fit(data_train_x, data_train_y)

## Tenendo outside air temp e torque_meas

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_OH_OT = make_pipeline(preprocessing_OH_OT, LogisticRegression())
log_reg_OH_OT.fit(data_train_x, data_train_y)

## Quantile transformation su power_avail

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_OH_QT = make_pipeline(preprocessing_OH_QT, LogisticRegression())
log_reg_OH_QT.fit(data_train_x, data_train_y)

## Feature engineering mean temp e  torq

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_OH_CH = make_pipeline(preprocessing_OH_CH, LogisticRegression())
log_reg_OH_CH.fit(data_train_x, data_train_y)

# Due quantile

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_OH_AQT = make_pipeline(preprocessing_OH_AQT, LogisticRegression())
log_reg_OH_AQT.fit(data_train_x, data_train_y)

# Valutazione performance al variare delle trasformazione nella pipeline

## Senza onehotencoding

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# Make predictions on the test set
y_pred = log_reg.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


Noto che le prestazioni sono basse, quindi cerco di agire sulla pipeline per migliorarle.

## Con onehotencoding

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# Make predictions on the test set
y_pred = log_reg_OH.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

 Noto un miglioramento su tutti gli  indici. È opportuno utilzzare onehotencoding

## Tenendo outside_air_temp e torque_meas

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



# Make predictions on the test set
y_pred = log_reg_OH_OT.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

 Non utilizzando la feature engineering su outside_air_temp e torque_meas si registra un migliroamento delle prestazioni.

## Quantile transformation su power_avail

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



# Make predictions on the test set
y_pred = log_reg_OH_QT.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

Noto un ulteriore miglioramento delle prestazioni a seguito del cambio del tipo di trasformazione effettuata su power_avail.

## Feature engineering di mean_temp e torq

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



# Make predictions on the test set
y_pred = log_reg_OH_CH.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

Noto un peggioramento dei vari indici a seguito dell'applicazione di una feature engineering alternativa.

In [ ]:
# prompt: calcola l'accurcay, precision, sensitivity e specificity della regressione logistica dopo aver elaborato i dati tramite pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



# Make predictions on the test set
y_pred = log_reg_OH_AQT.predict(data_test_x)

# Calculate the confusion matrix
cm = confusion_matrix(data_test_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
accuracy = accuracy_score(data_test_y, y_pred)
precision = precision_score(data_test_y, y_pred)
sensitivity = recall_score(data_test_y, y_pred) # Sensitivity is also known as Recall
# Specificity: TN / (TN + FP)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 # Handle potential division by zero

print("Confusion Matrix:")
print(cm)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

## Analisi dei risultati:

La trasformazione che è stata utilizzata in precedenza, che prevedeva la moltiplicazione di 'outside_air_temp' e 'torque_meas', non è efficace. Al contrario, ha avuto un esito positivo l'applicazione dello onehotencoding e il cambio di trasformazione per power_avail.

# Analisi delle prestazioni del quarto modello

## Cross validation

In [ ]:
# prompt: utilizza il cross_val_score per valutaeìre le performance di log_reg_OH_QT

from sklearn.model_selection import cross_val_score

# Evaluate the log_reg_OH_QT pipeline using cross-validation
# We use data_train_x and data_train_y for evaluation with cross-validation,
# as cross_val_score handles the splitting internally.
# Common scoring metrics for classification are 'accuracy', 'precision', 'recall', 'f1'.
# Let's evaluate using accuracy as an example.

# Use the pipeline log_reg_OH_QT directly in cross_val_score
# cross_val_score takes the estimator (your pipeline), the data, and the target.
# cv=... specifies the number of folds for cross-validation.
# scoring='accuracy' specifies the metric to use.
log_reg_OH_QT_scores_accuracy = cross_val_score(log_reg_OH_QT, data_train_x, data_train_y, cv=10, scoring="accuracy")

print("Cross-validation Accuracy Scores (log_reg_OH_QT):", log_reg_OH_QT_scores_accuracy)
print("Mean CV Accuracy (log_reg_OH_QT):", log_reg_OH_QT_scores_accuracy.mean())
print("Standard Deviation of CV Accuracy (log_reg_OH_QT):", log_reg_OH_QT_scores_accuracy.std())

# You can evaluate other metrics as well:
log_reg_OH_QT_scores_precision = cross_val_score(log_reg_OH_QT, data_train_x, data_train_y, cv=10, scoring="precision")
print("\nCross-validation Precision Scores (log_reg_OH_QT):", log_reg_OH_QT_scores_precision)
print("Mean CV Precision (log_reg_OH_QT):", log_reg_OH_QT_scores_precision.mean())

log_reg_OH_QT_scores_recall = cross_val_score(log_reg_OH_QT, data_train_x, data_train_y, cv=10, scoring="recall")
print("\nCross-validation Recall (Sensitivity) Scores (log_reg_OH_QT):", log_reg_OH_QT_scores_recall)
print("Mean CV Recall (log_reg_OH_QT):", log_reg_OH_QT_scores_recall.mean())

log_reg_OH_QT_scores_f1 = cross_val_score(log_reg_OH_QT, data_train_x, data_train_y, cv=10, scoring="f1")
print("\nCross-validation F1 Scores (log_reg_OH_QT):", log_reg_OH_QT_scores_f1)
print("Mean CV F1 (log_reg_OH_QT):", log_reg_OH_QT_scores_f1.mean())


Appliccando il cross-validation si ottengno degli indici pressochè identici per tutte le fold.

In [ ]:
# prompt: utilizza il cross_val_score per valutaeìre le performance di log_reg_OH_AQT

from sklearn.model_selection import cross_val_score

# Evaluate the log_reg_OH_AQT pipeline using cross-validation
# We use data_train_x and data_train_y for evaluation with cross-validation,
# as cross_val_score handles the splitting internally.
# Common scoring metrics for classification are 'accuracy', 'precision', 'recall', 'f1'.
# Let's evaluate using accuracy as an example.

# Use the pipeline log_reg_OH_AQT directly in cross_val_score
# cross_val_score takes the estimator (your pipeline), the data, and the target.
# cv=... specifies the number of folds for cross-validation.
# scoring='accuracy' specifies the metric to use.
log_reg_OH_AQT_scores_accuracy = cross_val_score(log_reg_OH_AQT, data_train_x, data_train_y, cv=10, scoring="accuracy")

print("Cross-validation Accuracy Scores (log_reg_OH_AQT):", log_reg_OH_AQT_scores_accuracy)
print("Mean CV Accuracy (log_reg_OH_AQT):", log_reg_OH_AQT_scores_accuracy.mean())
print("Standard Deviation of CV Accuracy (log_reg_OH_AQT):", log_reg_OH_AQT_scores_accuracy.std())

# You can evaluate other metrics as well:
log_reg_OH_AQT_scores_precision = cross_val_score(log_reg_OH_AQT, data_train_x, data_train_y, cv=10, scoring="precision")
print("\nCross-validation Precision Scores (log_reg_OH_AQT):", log_reg_OH_AQT_scores_precision)
print("Mean CV Precision (log_reg_OH_AQT):", log_reg_OH_AQT_scores_precision.mean())

log_reg_OH_AQT_scores_recall = cross_val_score(log_reg_OH_AQT, data_train_x, data_train_y, cv=10, scoring="recall")
print("\nCross-validation Recall (Sensitivity) Scores (log_reg_OH_AQT):", log_reg_OH_AQT_scores_recall)
print("Mean CV Recall (log_reg_OH_AQT):", log_reg_OH_AQT_scores_recall.mean())

log_reg_OH_AQT_scores_f1 = cross_val_score(log_reg_OH_AQT, data_train_x, data_train_y, cv=10, scoring="f1")
print("\nCross-validation F1 Scores (log_reg_OH_AQT):", log_reg_OH_AQT_scores_f1)
print("Mean CV F1 (log_reg_OH_AQT):", log_reg_OH_AQT_scores_f1.mean())


In [ ]:
log_reg_OPT= log_reg_OH_QT
preprocessing_OPT = preprocessing_OH_QT
data_train_processed_OPT = data_train_processed_OH_QT

## Confusion matrix

In [ ]:
# prompt: costruisci una confusion matrix di log_reg_OH con i colori

import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions on the test set using the log_reg_OH pipeline
y_pred_OPT = log_reg_OPT.predict(data_test_x)

# Calculate the confusion matrix for log_reg_OH
cm_OPT = confusion_matrix(data_test_y, y_pred_OPT)

# Plot the confusion matrix with colors
plt.figure(figsize=(8, 6))
sns.heatmap(cm_OPT, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Predicted Negative', 'Predicted Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Logistic Regression with One-Hot Encoding')
plt.show()


## Analisi coefficienti

In [ ]:
# prompt: definisci all_processed_feature_names come i nomi dei dati trasformati con la pipeline

all_processed_feature_names = data_train_processed_OPT.columns.tolist()

print("Nomi delle feature trasformate dalla pipeline OPT:")
all_processed_feature_names

In [ ]:
# prompt: printa i coefficienti della regressione logistica log_reg_OPT e le feature corrispondenti

# Get the Logistic Regression model from the pipeline
log_reg_model_OPT = log_reg_OPT.named_steps['logisticregression']

# Get the coefficients
coefficients_OPT = log_reg_model_OPT.coef_[0]

# Get the intercept
intercept_OPT = log_reg_model_OPT.intercept_[0]

# Print the coefficients and corresponding feature names
print("Intercept:", intercept_OPT)
print("\nCoefficients:")
for feature, coef in zip(all_processed_feature_names, coefficients_OPT):
    print(f"{feature}: {coef:.4f}")

Dall'analisi dei coefficienti si evince come le feature che hanno un impatto maggiore sull'output sono 'mean_gas_temp' e 'torque_meas'.

### p value

In [ ]:
# prompt: # prompt: analizza SOLO i p value associati a log_reg_OPT NON odds ratio

!pip install statsmodels
import statsmodels.api as sm

# Assicurati di avere i dati preprocessati per log_reg_OPT
# Questi dati sono stati creati in precedenza e sono in data_train_processed_OPT
# E la variabile target data_train_y

# Aggiungi una costante (intercetta) ai dati delle feature per statsmodels
X_train_sm_OPT = sm.add_constant(data_train_processed_OPT)

# Fit il modello di regressione logistica con statsmodels
logit_model_OPT = sm.Logit(data_train_y, X_train_sm_OPT)
result_OPT = logit_model_OPT.fit()

# Stampa il summary del modello per vedere i p-value
print(result_OPT.summary())

# Estrai i p-value
p_values_OPT = result_OPT.pvalues

print("\nP-values per log_reg_OPT:")
p_values_OPT


Dato che tutte le feature numeriche hanno un p-value minore di 0.05, c'è correlazione con l'output.

In [ ]:
# prompt: analizza l'odds ratio associati a log_reg_OPT

import numpy as np
# Analisi dell'Odds Ratio per log_reg_OPT

# Get the Logistic Regression model from the pipeline
log_reg_model_OPT = log_reg_OPT.named_steps['logisticregression']

# Get the coefficients
coefficients_OPT = log_reg_model_OPT.coef_[0]

# Get the feature names from the processed data
# This uses the function defined previously to get the correct feature names
all_processed_feature_names = to_dataframe(preprocessing_OPT, data_train_x).columns.tolist()

# Calculate Odds Ratios (exp(coefficient))
odds_ratios_OPT = np.exp(coefficients_OPT)

print("\nOdds Ratios (log_reg_OPT):")
for feature, odds in zip(all_processed_feature_names, odds_ratios_OPT):
    print(f"{feature}: {odds:.4f}")

# Note: Calculating p-values for individual coefficients in scikit-learn's
# LogisticRegression is not straightforward. scikit-learn is designed for
# prediction and performance, not statistical inference.
# To get p-values and more detailed statistical summaries (like those in statsmodels),
# you would typically need to use a library like statsmodels.

# If you needed to calculate p-values, you would generally do the following
# (but this requires fitting the model again using statsmodels):

# import statsmodels.api as sm
#
# # Get the processed training data (as a DataFrame from to_dataframe)
# data_train_processed_OPT_sm = to_dataframe(preprocessing_OPT, data_train_x)
#
# # Add a constant (intercept) term for statsmodels
# X_sm = sm.add_constant(data_train_processed_OPT_sm)
#
# # Fit the logistic regression model using statsmodels
# # Ensure y_target is the correct target variable (0 or 1)
# sm_log_reg_model = sm.Logit(data_train_y, X_sm).fit()
#
# # Print the summary which includes p-values
# print("\nStatsmodels Logistic Regression Summary:")
# print(sm_log_reg_model.summary())

# For this response, we focus on the Odds Ratios as requested and based on the
# available scikit-learn model fit. We'll mention the limitation regarding p-values.

print("\nAnalysis of Odds Ratios for log_reg_OPT:")
print("The Odds Ratio for a feature represents the change in the odds of the positive class (y=1) for a one-unit increase in that feature, holding all other features constant.")
print("- If Odds Ratio > 1: The odds of the positive class increase as the feature value increases.")
print("- If Odds Ratio < 1: The odds of the positive class decrease as the feature value increases.")
print("- If Odds Ratio ≈ 1: The feature has little effect on the odds.")
print("\nNote: Direct p-value calculation for coefficients is not directly available in scikit-learn's LogisticRegression. A library like statsmodels would be needed for this statistical inference.")


Dall'analisi del p-value risulta come le 4 feature numeriche risultino correlate all'uscita (p-value inferiore a 0.05).

C'è una correlazione molto forte con power_available

## Curva ROC

In [ ]:
# prompt: plotta la curva AUC

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Get the predicted probabilities for the positive class (class 1)
y_prob = log_reg_OPT.predict_proba(data_test_x)[:, 1]

# Calculate the false positive rate (fpr) and true positive rate (tpr) for various thresholds
fpr, tpr, thresholds = roc_curve(data_test_y, y_prob)

# Calculate the Area Under the Curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing') # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"Area Under the ROC Curve (AUC): {roc_auc:.4f}")


L'indice AUC della curva ROC è elevato.

## Lasso regression

In [ ]:
# prompt: prova ad usare la lasso regression per trovare la funzione di regressione logistica

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression



# Define the Logistic Regression model with Lasso (L1) penalty
log_reg_lasso_model = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42) # C=1.0 is a common starting point

# Integrate this model into the pipeline with OneHotEncoding
# We will replace the standard LogisticRegression step with the Lasso-regularized one.
log_reg_lasso_OPT_pipeline = make_pipeline(preprocessing_OPT, log_reg_lasso_model)

# Fit the pipeline to the training data
# Note: The preprocessing_OH pipeline is assumed to be defined and fitted already based on the preceding code.
# If you run this code block independently, make sure preprocessing_OH is available and fitted.
log_reg_lasso_OPT_pipeline.fit(data_train_x, data_train_y)

print("Lasso-regularized Logistic Regression pipeline fitted.")

# You can now use log_reg_lasso_OPT_pipeline for predictions and evaluation
# Make predictions on the test set
y_pred_lasso = log_reg_lasso_OPT_pipeline.predict(data_test_x)

# Evaluate the model (using confusion matrix, accuracy, etc. as done previously)
cm_lasso = confusion_matrix(data_test_y, y_pred_lasso)
tn_lasso, fp_lasso, fn_lasso, tp_lasso = cm_lasso.ravel()

accuracy_lasso = accuracy_score(data_test_y, y_pred_lasso)
precision_lasso = precision_score(data_test_y, y_pred_lasso)
sensitivity_lasso = recall_score(data_test_y, y_pred_lasso)
specificity_lasso = tn_lasso / (tn_lasso + fp_lasso) if (tn_lasso + fp_lasso) > 0 else 0

print("\nEvaluation Metrics for Lasso-regularized Logistic Regression:")
print("Confusion Matrix:")
print(cm_lasso)
print(f"\nAccuracy: {accuracy_lasso:.4f}")
print(f"Precision: {precision_lasso:.4f}")
print(f"Sensitivity (Recall): {sensitivity_lasso:.4f}")
print(f"Specificity: {specificity_lasso:.4f}")

# Analyze coefficients (Lasso regularization tends to push some coefficients to exactly zero)
log_reg_lasso_model_fitted = log_reg_lasso_OPT_pipeline.named_steps['logisticregression']

if hasattr(log_reg_lasso_model_fitted, 'coef_'):
    print("\nCoefficients of the Lasso-regularized Logistic Regression Model:")
    print("Coefficients:", log_reg_lasso_model_fitted.coef_[0])
    print("Intercept:", log_reg_lasso_model_fitted.intercept_[0])

    # To see which coefficient corresponds to which feature
    # We need the column names after preprocessing, which are in all_processed_feature_names
    if len(log_reg_lasso_model_fitted.coef_[0]) == len(all_processed_feature_names):
        coefficients_lasso_df = pd.DataFrame({
            'Feature': all_processed_feature_names,
            'Coefficient': log_reg_lasso_model_fitted.coef_[0]
        })
        # Sort by absolute coefficient value to see the most important features
        coefficients_lasso_df['Abs_Coefficient'] = np.abs(coefficients_lasso_df['Coefficient'])
        coefficients_lasso_df_sorted = coefficients_lasso_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns='Abs_Coefficient')

        print("\nFeature Coefficients (Lasso, Sorted by Absolute Value):")
        print(coefficients_lasso_df_sorted)

        # Count how many coefficients are exactly zero
        zero_coefficients_count = (coefficients_lasso_df['Coefficient'] == 0).sum()
        print(f"\nNumber of coefficients exactly zero (due to L1 regularization): {zero_coefficients_count}")

    else:
        print("\nWarning: Number of coefficients does not match the number of processed features.")

# Plot ROC Curve for Lasso model
y_prob_lasso = log_reg_lasso_OPT_pipeline.predict_proba(data_test_x)[:, 1]
fpr_lasso, tpr_lasso, thresholds_lasso = roc_curve(data_test_y, y_prob_lasso)
roc_auc_lasso = auc(fpr_lasso, tpr_lasso)

plt.figure(figsize=(8, 6))
plt.plot(fpr_lasso, tpr_lasso, color='darkorange', lw=2, label=f'ROC curve (Lasso) (AUC = {roc_auc_lasso:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Lasso-regularized Logistic Regression')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"\nArea Under the ROC Curve (AUC) for Lasso: {roc_auc_lasso:.4f}")


Peggiora le prestazioni.

## Ridge regression

In [ ]:
# prompt: Applica la ridge regression

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Define the Logistic Regression model with Ridge (L2) penalty
log_reg_ridge_model = LogisticRegression(penalty='l2', solver='liblinear', C=1.0, random_state=42) # C=1.0 is a common starting point, solver='liblinear' works with l2

# Integrate this model into the pipeline with OneHotEncoding and chosen feature engineering
# We will use the preprocessing_OPT pipeline which performed better in the previous analysis.
# Replace the standard LogisticRegression step with the Ridge-regularized one.
log_reg_ridge_OPT_pipeline = make_pipeline(preprocessing_OPT, log_reg_ridge_model)

# Fit the pipeline to the training data
log_reg_ridge_OPT_pipeline.fit(data_train_x, data_train_y)

print("Ridge-regularized Logistic Regression pipeline fitted using preprocessing_OPT.")

# You can now use log_reg_ridge_OPT_pipeline for predictions and evaluation
# Make predictions on the test set
y_pred_ridge = log_reg_ridge_OPT_pipeline.predict(data_test_x)

# Evaluate the model (using confusion matrix, accuracy, etc. as done previously)
cm_ridge = confusion_matrix(data_test_y, y_pred_ridge)
tn_ridge, fp_ridge, fn_ridge, tp_ridge = cm_ridge.ravel()

accuracy_ridge = accuracy_score(data_test_y, y_pred_ridge)
precision_ridge = precision_score(data_test_y, y_pred_ridge)
sensitivity_ridge = recall_score(data_test_y, y_pred_ridge)
specificity_ridge = tn_ridge / (tn_ridge + fp_ridge) if (tn_ridge + fp_ridge) > 0 else 0

print("\nEvaluation Metrics for Ridge-regularized Logistic Regression:")
print("Confusion Matrix:")
print(cm_ridge)
print(f"\nAccuracy: {accuracy_ridge:.4f}")
print(f"Precision: {precision_ridge:.4f}")
print(f"Sensitivity (Recall): {sensitivity_ridge:.4f}")
print(f"Specificity: {specificity_ridge:.4f}")

# Analyze coefficients (Ridge regularization shrinks coefficients towards zero but doesn't make them exactly zero)
log_reg_ridge_model_fitted = log_reg_ridge_OPT_pipeline.named_steps['logisticregression']

if hasattr(log_reg_ridge_model_fitted, 'coef_'):
    print("\nCoefficients of the Ridge-regularized Logistic Regression Model:")
    # Use all_processed_feature_names obtained from the previous cell
    if len(log_reg_ridge_model_fitted.coef_[0]) == len(all_processed_feature_names):
        coefficients_ridge_df = pd.DataFrame({
            'Feature': all_processed_feature_names,
            'Coefficient': log_reg_ridge_model_fitted.coef_[0]
        })
        # Sort by absolute coefficient value
        coefficients_ridge_df['Abs_Coefficient'] = np.abs(coefficients_ridge_df['Coefficient'])
        coefficients_ridge_df_sorted = coefficients_ridge_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns='Abs_Coefficient')

        print("\nFeature Coefficients (Ridge, Sorted by Absolute Value):")
        print(coefficients_ridge_df_sorted)

    else:
        print("\nWarning: Number of coefficients does not match the number of processed features.")
        print("Number of coefficients:", len(log_reg_ridge_model_fitted.coef_[0]))
        print("Number of processed features:", len(all_processed_feature_names))

else:
    print("The fitted model does not have 'coef_' attribute.")


# Plot ROC Curve for Ridge model
y_prob_ridge = log_reg_ridge_OPT_pipeline.predict_proba(data_test_x)[:, 1]
fpr_ridge, tpr_ridge, thresholds_ridge = roc_curve(data_test_y, y_prob_ridge)
roc_auc_ridge = auc(fpr_ridge, tpr_ridge)

plt.figure(figsize=(8, 6))
plt.plot(fpr_ridge, tpr_ridge, color='darkgreen', lw=2, label=f'ROC curve (Ridge) (AUC = {roc_auc_ridge:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Ridge-regularized Logistic Regression')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"\nArea Under the ROC Curve (AUC) for Ridge: {roc_auc_ridge:.4f}")


Sia la ridge che la lasso regression peggiorano le prestazioni. Questo è dovuto al fatto che tutte le feature sono determinanti per predire lo stato di salute.

## Analisi risultati

La lasso regression peggiora le performance

## Tuning dei parametri

In [ ]:
# prompt: esegui un'operazione di fine tuning sui parametri di log_reg_OPT

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to tune for the Logistic Regression step
# The parameter names in the grid must be prefixed with the name of the step in the pipeline ('logisticregression')
# followed by a double underscore '__' and the parameter name.
param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100], # Inverse of regularization strength; smaller values specify stronger regularization.
    'logisticregression__penalty': ['l2'], # Regularization type. 'l1' can also be used, but requires 'liblinear' solver for some versions. 'l2' is standard.
    'logisticregression__solver': ['lbfgs', 'liblinear'], # Algorithm to use in the optimization problem. 'liblinear' is good for small datasets, 'lbfgs' is generally faster for larger ones and supports 'l2'.
    'logisticregression__max_iter': [1000, 2000, 3000] # Maximum number of iterations for the solver to converge.
}

# Create a GridSearchCV object
# estimator: Your pipeline (log_reg_OPT)
# param_grid: The dictionary of parameters to tune
# cv: Number of cross-validation folds (e.g., 5 or 10)
# scoring: The metric to optimize (e.g., 'accuracy', 'precision', 'recall', 'f1', 'roc_auc')
# n_jobs: Number of CPU cores to use (-1 means all available cores)
# verbose: Controls the verbosity: higher values mean more output during the process.
grid_search = GridSearchCV(estimator=log_reg_OPT,
                           param_grid=param_grid,
                           cv=5, # Using 5 folds for cross-validation
                           scoring='roc_auc', # Optimizing for AUC, as it considers the trade-off between sensitivity and specificity
                           n_jobs=-1, # Use all available cores
                           verbose=2) # Print progress messages

# Fit GridSearchCV on the training data (features and target)
# GridSearchCV will automatically apply the pipeline's preprocessing steps within each fold.
grid_search.fit(data_train_x, data_train_y)

# Get the best parameters found by GridSearchCV
print("Best parameters found by GridSearchCV:")
print(grid_search.best_params_)

# Get the best estimator (the pipeline with the best parameters)
best_log_reg_OPT = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_best = best_log_reg_OPT.predict(data_test_x)
y_prob_best = best_log_reg_OPT.predict_proba(data_test_x)[:, 1]

# Calculate and print metrics for the best model on the test set
cm_best = confusion_matrix(data_test_y, y_pred_best)
tn_best, fp_best, fn_best, tp_best = cm_best.ravel()

accuracy_best = accuracy_score(data_test_y, y_pred_best)
precision_best = precision_score(data_test_y, y_pred_best)
sensitivity_best = recall_score(data_test_y, y_pred_best)
specificity_best = tn_best / (tn_best + fp_best) if (tn_best + fp_best) > 0 else 0
roc_auc_best = auc(roc_curve(data_test_y, y_prob_best)[0], roc_curve(data_test_y, y_prob_best)[1])


print("\nMetrics for the Best Model (after GridSearchCV) on the Test Set:")
print("Confusion Matrix:")
print(cm_best)
print(f"\nAccuracy: {accuracy_best:.4f}")
print(f"Precision: {precision_best:.4f}")
print(f"Sensitivity (Recall): {sensitivity_best:.4f}")
print(f"Specificity: {specificity_best:.4f}")
print(f"Area Under the ROC Curve (AUC): {roc_auc_best:.4f}")

# Optional: Plot Confusion Matrix and ROC Curve for the best model
plt.figure(figsize=(8, 6))
sns.heatmap(cm_best, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Predicted Negative', 'Predicted Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Best Logistic Regression Model (after GridSearchCV)')
plt.show()

fpr_best, tpr_best, thresholds_best = roc_curve(data_test_y, y_prob_best)
plt.figure(figsize=(8, 6))
plt.plot(fpr_best, tpr_best, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_best:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Best Logistic Regression Model')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


Ci mette 4 minuti

Il tuning dei parametri non ha migliorato gli indici. Ha comportato un lieve peggioramento.

## Randomized Search

In [ ]:
# prompt: utilizza RandomsizedSearchcV

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'logisticregression__C': np.logspace(-3, 2, 6),
    'logisticregression__penalty': ['l2'],
    'logisticregression__solver': ['lbfgs', 'liblinear'],
    'logisticregression__max_iter': [1000, 2000, 3000, 5000],
}


random_search = RandomizedSearchCV(estimator=log_reg_OPT,
                                   param_distributions=param_dist,
                                   n_iter=48,
                                   cv=3,
                                   scoring='roc_auc',
                                   n_jobs=-1,
                                   random_state=42,
                                   verbose=2)


random_search.fit(data_train_x, data_train_y)


print("Best parameters found by RandomizedSearchCV:")
print(random_search.best_params_)




In [ ]:
final_model = random_search.best_estimator_


## Valuto la performance del nuovo modello

In [ ]:
# prompt: valuta le performance di final model su test

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
# The code for evaluating the final_model on the test set is already present in the preceding code.
# You just need to run the last few cells of the provided code block.

# Evaluate the final_model (which is the best_estimator_ from RandomizedSearchCV) on the test set
y_pred_final = final_model.predict(data_test_x)
y_prob_final = final_model.predict_proba(data_test_x)[:, 1]

# Calculate and print metrics for the final_model on the test set
cm_final = confusion_matrix(data_test_y, y_pred_final)
tn_final, fp_final, fn_final, tp_final = cm_final.ravel()

accuracy_final = accuracy_score(data_test_y, y_pred_final)
precision_final = precision_score(data_test_y, y_pred_final)
sensitivity_final = recall_score(data_test_y, y_pred_final)
specificity_final = tn_final / (tn_final + fp_final) if (tn_final + fp_final) > 0 else 0
roc_auc_final = auc(roc_curve(data_test_y, y_prob_final)[0], roc_curve(data_test_y, y_prob_final)[1])

print("\nMetrics for the Final Model (from RandomizedSearchCV) on the Test Set:")
print("Confusion Matrix:")
print(cm_final)
print(f"\nAccuracy: {accuracy_final:.4f}")
print(f"Precision: {precision_final:.4f}")
print(f"Sensitivity (Recall): {sensitivity_final:.4f}")
print(f"Specificity: {specificity_final:.4f}")
print(f"Area Under the ROC Curve (AUC): {roc_auc_final:.4f}")

# Optional: Plot Confusion Matrix and ROC Curve for the final model
plt.figure(figsize=(8, 6))
sns.heatmap(cm_final, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Predicted Negative', 'Predicted Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Final Logistic Regression Model (after RandomizedSearchCV)')
plt.show()

fpr_final, tpr_final, thresholds_final = roc_curve(data_test_y, y_prob_final)
plt.figure(figsize=(8, 6))
plt.plot(fpr_final, tpr_final, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_final:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Final Logistic Regression Model')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

Noto un peggioramento delle prestazioni